# Intro to predictive coding, making an image more huffable 
Joshua Stough, DIP 8.2.1 and 8.2.9


In [1]:
%matplotlib widget
import matplotlib.pyplot as plt
import scipy.ndimage as ndimage
import numpy as np

from scipy.ndimage import correlate
from skimage.filters import *
from scipy.stats import entropy

In [2]:
# utility: what is arr?
def arrInfo(arr):
    return arr.shape, arr.min(), arr.max(), arr.dtype

In [3]:
I = plt.imread('germanTown.jpg')

In [4]:
arrInfo(I)

((363, 600, 3), 0, 255, dtype('uint8'))

# Reduce to single channel
For the sake of simplicity in the types and dimensions, let's reduce to 2D

In [5]:
# https://scikit-image.org/docs/dev/auto_examples/color_exposure/plot_rgb_to_gray.html
from skimage.color import rgb2gray
I = rgb2gray(I)
arrInfo(I)

((363, 600), 0.0, 1.0, dtype('float64'))

In [6]:
# Let's bring it back to 0-255 uint8
I = (255*I).astype(np.uint8)
arrInfo(I)

((363, 600), 0, 255, dtype('uint8'))

In [7]:
plt.figure()
plt.imshow(I, cmap='gray', interpolation=None)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Now, let's do histograms

In [25]:
bins = np.arange(257)
freq, bins = np.histogram(I.ravel(), bins=bins)

In [26]:
entropy(freq)

5.192338624859175

In [10]:
plt.figure()
plt.bar(bins[:-1], freq)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<BarContainer object of 255 artists>

# Spatial Filtering
We'll make a filter for the current pixel - the last pixel.

In [11]:
h = np.array([-1, 1], ndmin=2).astype(np.int16)

In [12]:
arrInfo(h)

((1, 2), -1, 1, dtype('int16'))

In [13]:
h

array([[-1,  1]], dtype=int16)

# Filter the image based on h

In [49]:
If = correlate(I.astype(np.int16), h, mode='constant', cval=0)

In [50]:
arrInfo(If)

((363, 600), -183, 184, dtype('int16'))

In [51]:
f, ax = plt.subplots(1,2, figsize=(8,3), sharex=True, sharey=True)

ax[0].imshow(I, cmap='gray', interpolation=None)
ax[0].set_title('Original')
ax[1].imshow(If, cmap='gray', interpolation=None)#, vmin=-255, vmax=255)
ax[1].set_title('Filtered')

plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Let's look at the histograms of both the original and the filtered images

In [52]:
f, ax = plt.subplots(1,2,figsize=(8,3))
ax[0].hist(I.ravel(), bins=256);
ax[1].hist(If.ravel(), bins=np.arange(-200,200,1));

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# How about the entropy of these two versions

In [53]:
I_hist, I_bins = np.histogram(I.ravel(), bins=np.arange(257))
If_hist, If_bins = np.histogram(If.ravel(), bins=np.arange(-255,256,1))

In [54]:
entropy(I_hist, base=2)

7.490961184700552

In [55]:
entropy(If_hist, base=2)

5.711917681996355

# But can we reconstruct the original from If?


In [56]:
# Just check to see the first columns are equal
np.all(np.equal(I[:,0], If[:,0]))

True

In [61]:
def recon(I):
    If = I.copy()
    for i in range(1,If.shape[1]):
        If[:,i] = If[:,i-1] + If[:,i]
    return If

In [63]:

plt.figure()
plt.imshow(recon(If), cmap='hot')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …